# Árvores II - Tarefa 3

### 1. Carregar as bases

Vamos utilizar nesta tarefa as bases de reconhecimento de atividade humana através do celular. Carregue novamente as bases salvas na tarefa I.

In [35]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, KFold, cross_val_predict
import plotly.graph_objs as go

In [9]:
X_train = pd.read_excel("/content/X_train.xlsx")[["tGravityAcc-min()-X", "fBodyAcc-mad()-X", "tGravityAcc-mean()-Y"]]
X_test = pd.read_excel("/content/X_test.xlsx")[["tGravityAcc-min()-X", "fBodyAcc-mad()-X", "tGravityAcc-mean()-Y"]]

In [10]:
y_train = pd.read_excel("/content/y_train.xlsx").drop(columns=["Unnamed: 0"])
y_test = pd.read_excel("/content/y_test.xlsx").drop(columns=["Unnamed: 0"])

### 2. Calcule os ```ccp_alphas```.

Vamos seguir uma lógica bem em linha com o que já estamso acostumados, com as seguintes orientações:

- Utilizar treinamento e teste conforme já vieram definidos originalmente
- Por pragmatismo, utilizar ```min_samples_leaf=20```
- Utilize as mesmas "3 melhores variáveis" identificadas no exercício anterior.

In [14]:
%%time
tree = DecisionTreeClassifier(random_state=42, min_samples_leaf=20).cost_complexity_pruning_path(X_train, y_train)
ccp_alphas, impurities = tree.ccp_alphas, tree.impurities

CPU times: user 33.8 ms, sys: 0 ns, total: 33.8 ms
Wall time: 41.3 ms


In [18]:
ccp_alphas[:5]

array([0.00000000e+00, 6.59906426e-06, 7.04310757e-06, 8.16104461e-06,
       8.62549431e-06])

### 3. *Grid Search*

Vou deixar especificações iniciais mínimas, que visam limitar o tempo de máquina, pois um procedimento desses pode demorar muito tempo dependendo da especificação. Mas conforme você for ficando confortável com o tempo consumido pelo procedimento, pode fazer um algoritmo mais exaustivo, por exemplo, avaliando mais valores de ```ccp_alpha```.

- Meça o tempo
- Utilize a base de treinamento
- Utilize um *cross validation* do tipo *k-fold*, especifique k=10
- Você pode ler 1 a cada ```k``` valores para uma melhor varredura utilizando, por exemplo, ```ccp_alpha[::10]```
- Não se esqueça de limitar o número de variáveis

In [19]:
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'ccp_alpha': ccp_alphas[:5]
}

In [20]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [21]:
%%time
grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=kf, scoring='accuracy').fit(X_train, y_train)

CPU times: user 1min 57s, sys: 199 ms, total: 1min 58s
Wall time: 2min


In [22]:
best_params = grid_search.best_params_

In [23]:
best_params

{'ccp_alpha': 8.625494307173812e-06,
 'criterion': 'entropy',
 'max_depth': None,
 'min_samples_leaf': 2,
 'min_samples_split': 5}

### 4. Avaliando a árvore

- Obtenha a árvore na melhor configuração treinada em toda a base de treino
- Calcule a acurácia dessa árvore na base de testes
- Visualize a matriz de confusão

In [24]:
best_tree = grid_search.best_estimator_

In [26]:
y_pred = best_tree.predict(X_test)

In [28]:
accuracy = accuracy_score(y_test, y_pred)

In [29]:
accuracy

0.7030878859857482

In [30]:
conf_matrix = confusion_matrix(y_test, y_pred)

In [32]:
heatmap = go.Heatmap(
  z=conf_matrix,
  x=['Predito: ' + str(i) for i in range(conf_matrix.shape[1])],
  y=['Real: ' + str(i) for i in range(conf_matrix.shape[0])],
  colorscale='Viridis'
)


layout = go.Layout(
    title='Matriz de Confusão',
    xaxis=dict(title='Predito'),
    yaxis=dict(title='Real')
)


fig = go.Figure(data=[heatmap], layout=layout)

In [33]:
fig.show()

### 5. Melhorando a árvore

A melhor forma de se melhorar um algoritmo é colocando nele novas variáveis que agreguem valor. Podemos usar a força-bruta e ir colocando variáveis aleatoriamente - ou colocar todas e deixar rodando por um bom tempo - ou utilizar uma lógica eficiente e fazer uma seleção de variáveis.

- Observe que há classes mais fáceis e mais difíceis de se identificar
- Crie uma variável binária para uma das classes de maior erro
- Fala uma árvore de classificação bem simples para esta variável:
    - utilize ```mean_samples_leaf=20```
    - utilize ```max_depth=4```
    - coloque todas as variáveis
- Observe a importância das variáveis, e selecione as 3 com maior importância
- Rode novamente o algoritmo acima com as 3 novas variáveis e avalie a acurácia

In [85]:
# Concatenar a base de treino e teste para facilitar a categorização das variáveis
X = pd.concat([X_train, X_test])
y = pd.concat([y_train, y_test])

In [80]:
# Calcular o erro padrão

std_err = np.std(X, axis=0) / np.sqrt(len(X))

for i, std_err_var in enumerate(std_err):
    print(f"{X_train.columns[i]}: {std_err_var}")

# A variável de maior erro é tGravityAcc-min()-X

tGravityAcc-min()-X: 0.0049961273921848666
fBodyAcc-mad()-X: 0.004506281938510973
tGravityAcc-mean()-Y: 0.0037335034196098924


In [81]:
# Categorizar a variável com mairo erro padrão

X_dummies = pd.get_dummies(X["tGravityAcc-min()-X"], prefix="categoria")
X_dummies = pd.concat([X, X_dummies], axis=1)
X_dummies.drop("tGravityAcc-min()-X", axis=1, inplace=True)

In [86]:
X_train, X_test, y_train, y_test = train_test_split(X_dummies, y, test_size=0.2, random_state=42)

In [87]:
tree = DecisionTreeClassifier(min_samples_leaf=20, max_depth=4, random_state=42).fit(X_train, y_train)

In [88]:
indices = tree.feature_importances_.argsort()[-3:][::-1]

In [89]:
best_variables = X_train.columns[indices]

In [90]:
improved_X_train = X_train[best_variables]

In [93]:
tree = DecisionTreeClassifier(min_samples_leaf=20, max_depth=4, random_state=42).fit(improved_X_train, y_train)

In [95]:
y_pred = tree.predict(X_test[best_variables])

In [97]:
accuracy = accuracy_score(y_test, y_pred)

In [98]:
accuracy

0.7631067961165049